In the following we use R packages to work through some DAG examples that illustrate features of _structural causal models_.

First, import required Python:

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import pandas as pd
import numpy as np

In [3]:
# This will probably just need to be commented out.  It's a kluge for my environ
# import os
# os.environ['R_HOME']='/home/lynd/anaconda36/lib/R'

In [4]:
import rpy2
rpy2.__version__
from rpy2.rinterface import R_VERSION_BUILD
print(R_VERSION_BUILD)

('3', '4.3', '', 73796)


In [5]:
import rpy2.robjects as robjects

In [6]:
from rpy2.robjects.packages import importr
base=importr('base')
utils=importr('utils')
stats=importr('stats')
rgrDevs=importr('grDevices')

There are a couple of R packages that can be used to define and evaluate DAGs.  These include `dagitty`, `dagR`, and `ggm`.   Here we're going to start with `dagR` to specify some basic types of causal graphs.  Personally I find `daggity` easier, so we'll move on to it quickly.

In [7]:
rdagR=importr('dagR')

In his seminal work about path analysis, Wright(1921) defined three basic kinds of directed graph structures:  

\begin{align}
Z \to X \to Y \\
X \leftarrow Z \to Y \\
X \to Z \leftarrow Y
\end{align}  

Let's do each of these using `dagR`.  First, lets try a couple of built-in demos to make sure things are working.

In [8]:
dagDemo1=rdagR.demo_dag1()

In [9]:
dagDemoDraw=rdagR.dag_draw(dagDemo1)

OK?  Got a graph?  So, now let's set up a $X \to X \to Y$ graph, eqn 1, above. First we'll close the graph we just made.

In [32]:
rgrDevs.graphics_off() # close the graph window

rpy2.rinterface.NULL

In this next bit of code you'll notice the use of `robjects` methods to define string and int R vectors.

In [12]:
dag1=rdagR.dag_init(y_name='Y',x_name="X",  # labels for Y and X
    cov_names=robjects.StrVector(['Z']),    # define a covariate Z
    covs=robjects.IntVector([1]),           # specify an observed covariate
    arcs=robjects.IntVector([0,-1,1,0]))    # specify "arcs" (arrows) betw vars

Let's take a look at the graph:

In [13]:
dag1Graph=rdagR.dag_draw(dag1,noxy=2)

The default doesn't put Z, X, and Y in a "chain," but it's the graph we want. Input parameters in the key/value arguments are a little obtuse. Here's how you can get R help about the `dagR` package:

In [16]:
import rpy2.robjects.help as rh
dagR_help=rh.Package('dagR')

In [17]:
rInit=dagR_help.fetch('dag.init')
rInit.sections.keys()
print(rInit.to_docstring(('description',)))

description
-----------


 Allows setting up a new DAG.
  See the  demo.dag0  to  demo.dag6  functions for some example specifications.
 




It can be a little cumbersome getting R documentation out through rpy2.  An alternative is to look for it online.

In the specification of dag1Graph, above, the vector in the "arcs" argument defines arrows between variables using pairs of variable (node) identifiers. 0 is the Tx or exposure variable X, and -1 is the outcome variable Y.  Z is 1.   So [0,-1, 1 0] means that there's an arrow from X to Y, and an arrow from Z to X.

**EXERCISE**  Modify the above code to produce the other two graphic structures.  

_Questions_ Which of the structures has been called a _collider_?  Which one a _fork_?

### Adjusting for Z

So far all we've done is to make a couple of graphs using X, Y and Z.  Let's add an edge to our DAG.

In [39]:
addedArc=robjects.IntVector([2,3])
dag2=rdagR.add_arc(dag1,arc=addedArc)

In [40]:
dag2Graph=rdagR.dag_draw(dag2)

So, does adjusting for Z help in estimating $X \to Y$?

In [41]:
dag2AdjZ=rdagR.dag_adjust(dag2,A=2)
#A=2 indicates Z as the covariate to adjust for

In [42]:
dag2AdGraph=rdagR.dag_draw(dag2AdjZ)

**EXERCISE**  Create the graph $X \to X \leftarrow Y$.   check to see what happens if you adjust for Z.

**EXERCISE**  Come up with a model of your own that has _two_ covariates, Z1 and Z2, and that has one or more chains (eqn 1, above), a fork (eqn 2), and a collider (eqn 3).   Try to imagine what the effects might be of adjusting for either Z1 or Z2.

The `dagR` package is rather cumbersome to use, even in R.  But it does have some useful functionality.  

Next stop: `dagitty` for DAGs.